In [17]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import scipy.stats as stats
from sklearn.model_selection import train_test_split, KFold, cross_val_score
from sklearn.metrics import r2_score
from sklearn.preprocessing import StandardScaler, PolynomialFeatures
from sklearn.linear_model import Ridge, Lasso, ElasticNet, LinearRegression, RidgeCV, LassoCV, ElasticNetCV
from sklearn.model_selection import cross_val_score
import statsmodels.api as sm
from statsmodels.sandbox.regression.predstd import wls_prediction_std
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import mean_squared_error

%matplotlib inline
sns.set_style('whitegrid')
plt.style.use('fivethirtyeight')

In [3]:
df = pd.read_csv('/Users/jonathanbeatty/Desktop/DataSets/Project_2/train.csv')

test_df = pd.read_csv('/Users/jonathanbeatty/Desktop/DataSets/Project_2/test.csv')

In [4]:
df.drop('Pool QC', axis = 1, inplace= True)
df.drop('Misc Feature', axis = 1, inplace = True)
df.drop('Fence', axis = 1, inplace = True)
df.drop('Alley', axis = 1, inplace = True)
df.drop('Lot Frontage', axis = 1, inplace = True)
df.drop('Bsmt Qual', axis = 1, inplace = True)
df.drop('Bsmt Cond', axis = 1, inplace = True)
df.drop('Bsmt Exposure', axis = 1, inplace = True)
df.drop('BsmtFin Type 1', axis = 1, inplace = True)
df.drop('BsmtFin Type 2', axis = 1, inplace = True)
df.drop('Electrical', axis = 1, inplace = True)
df.drop('Fireplace Qu', axis = 1, inplace = True)
df.drop('Garage Type', axis = 1, inplace = True)
df.drop('Garage Yr Blt', axis = 1, inplace = True)
df.drop('Garage Finish', axis = 1, inplace = True)
df.drop('Garage Qual', axis = 1, inplace = True)
df.drop('Garage Cond', axis = 1, inplace = True)
df.drop('Mas Vnr Area', axis = 1, inplace = True)

In [5]:
test_df.drop('Pool QC', axis = 1, inplace= True)
test_df.drop('Misc Feature', axis = 1, inplace = True)
test_df.drop('Fence', axis = 1, inplace = True)
test_df.drop('Alley', axis = 1, inplace = True)
test_df.drop('Lot Frontage', axis = 1, inplace = True)
test_df.drop('Bsmt Qual', axis = 1, inplace = True)
test_df.drop('Bsmt Cond', axis = 1, inplace = True)
test_df.drop('Bsmt Exposure', axis = 1, inplace = True)
test_df.drop('BsmtFin Type 1', axis = 1, inplace = True)
test_df.drop('BsmtFin Type 2', axis = 1, inplace = True)
test_df.drop('Electrical', axis = 1, inplace = True)
test_df.drop('Fireplace Qu', axis = 1, inplace = True)
test_df.drop('Garage Type', axis = 1, inplace = True)
test_df.drop('Garage Yr Blt', axis = 1, inplace = True)
test_df.drop('Garage Finish', axis = 1, inplace = True)
test_df.drop('Garage Qual', axis = 1, inplace = True)
test_df.drop('Garage Cond', axis = 1, inplace = True)
test_df.drop('Mas Vnr Area', axis = 1, inplace = True)

In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2051 entries, 0 to 2050
Data columns (total 63 columns):
Id                 2051 non-null int64
PID                2051 non-null int64
MS SubClass        2051 non-null int64
MS Zoning          2051 non-null object
Lot Area           2051 non-null int64
Street             2051 non-null object
Lot Shape          2051 non-null object
Land Contour       2051 non-null object
Utilities          2051 non-null object
Lot Config         2051 non-null object
Land Slope         2051 non-null object
Neighborhood       2051 non-null object
Condition 1        2051 non-null object
Condition 2        2051 non-null object
Bldg Type          2051 non-null object
House Style        2051 non-null object
Overall Qual       2051 non-null int64
Overall Cond       2051 non-null int64
Year Built         2051 non-null int64
Year Remod/Add     2051 non-null int64
Roof Style         2051 non-null object
Roof Matl          2051 non-null object
Exterior 1st       20

In [7]:
test_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 879 entries, 0 to 878
Data columns (total 62 columns):
Id                 879 non-null int64
PID                879 non-null int64
MS SubClass        879 non-null int64
MS Zoning          879 non-null object
Lot Area           879 non-null int64
Street             879 non-null object
Lot Shape          879 non-null object
Land Contour       879 non-null object
Utilities          879 non-null object
Lot Config         879 non-null object
Land Slope         879 non-null object
Neighborhood       879 non-null object
Condition 1        879 non-null object
Condition 2        879 non-null object
Bldg Type          879 non-null object
House Style        879 non-null object
Overall Qual       879 non-null int64
Overall Cond       879 non-null int64
Year Built         879 non-null int64
Year Remod/Add     879 non-null int64
Roof Style         879 non-null object
Roof Matl          879 non-null object
Exterior 1st       879 non-null object
Exteri

In [8]:
df.fillna(0.0,inplace=True)

In [9]:
test_df.fillna(0.0,inplace=True)

In [10]:
# features = [each for each in df_numeric.columns if each != 'SalePrice']
features = ['Lot Area', 'Overall Qual',
             'Year Built', 'Year Remod/Add',
            '1st Flr SF', 'Gr Liv Area', 'Full Bath', 'Garage Cars', 'Garage Area', 
            'Total Bsmt SF', '1st Flr SF', 'TotRms AbvGrd']
# features_test = [each for each in test_df_numeric.columns if each != 'SalePrice']
features_test =['Lot Area', 'Overall Qual',
             'Year Built', 'Year Remod/Add',
            '1st Flr SF', 'Gr Liv Area', 'Full Bath', 'Garage Cars', 'Garage Area', 
            'Total Bsmt SF', '1st Flr SF', 'TotRms AbvGrd']

X = df[features]
y = df['SalePrice']
X_test = test_df[features_test]
# kaggle_n = kaggle_new[common_to_both]
# X_n = X[common_to_both]

In [11]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

In [12]:
y_test.shape

(513,)

In [13]:
y_test.shape

(513,)

## LASSO REGRESSION MODELING

In [14]:
# kaggle_n = kaggle_new[common_to_both]
# X_n = X[common_to_both]

# ss = StandardScaler()
# ss.fit(X_n)
# X_n_s = ss.transform(X_n)

# kaggle_n_s = ss.transform(kaggle_n)

# lcv = LassoCV(alphas = np.linspace(0.1,5,50), cv = 10)

# lcv.fit(X_n_s, y)
# print(lcv.score(X_n_s, y))
# predictions = lcv.predict(kaggle_n_s)

In [19]:
pipe = Pipeline([
    ()
    ('ss', StandardScaler()),
    ('lcv', LassoCV(n_alphas=500))
])

pipe.fit(X_train, y_train)

Pipeline(memory=None,
     steps=[('ss', StandardScaler(copy=True, with_mean=True, with_std=True)), ('lcv', LassoCV(alphas=None, copy_X=True, cv=None, eps=0.001, fit_intercept=True,
    max_iter=1000, n_alphas=500, n_jobs=1, normalize=False, positive=False,
    precompute='auto', random_state=None, selection='cyclic', tol=0.0001,
    verbose=False))])

In [21]:
print(pipe.score(X_train, y_train))
print(pipe.score(X_test, y_test))

0.7762947549468021
0.8350676369334137


In [ ]:
# params_grid = {
#     'ss__with_mean': [True, False],
#     'ss__with_std': [True, False],
# }

# gs = GridSearchCV(pipe, params_grid, n_jobs=-1, verbose =2, cv=7)

In [ ]:
gs.fit(X_train, y_train)

In [ ]:
gs.best_estimator_

In [ ]:
gs.best_score_

In [ ]:
gs.best_params_

In [ ]:
gs.score(X_train, y_train)

In [ ]:
gs.score(X_test, y_test)

In [ ]:
X_test_num[features_test].head()

In [ ]:
X_train.shape

In [ ]:
X_test_num[features_test].shape

In [ ]:
predictions = gs.predict(kaggle_n)

In [ ]:
len(predictions)

In [ ]:
pred_dic = {'Id': test_df['Id'],
            'SalePrice': predictions
           }
pred_dic

preds_df = pd.DataFrame(data=pred_dic)
preds_df.info()

preds_df.head()

In [ ]:
preds_df[['Id', 'SalePrice']].to_csv('kaggle_preds_7.csv', index=False)#without this it adds unnamed:0